# Lecture 11: Approximation Methods and Root Finding

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import scipy.interpolate
import pylab
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML
from ipywidgets import interact, interactive, fixed, interact_manual
from matplotlib import animation, rc
%matplotlib inline


## Part 1: Interpolation
-------------

Given a set of *N* points $(x_i, y_i)$ with $i = 1, 2, …N$, we sometimes need a function $\hat{f}(x)$ which returns $y_i = f(x_i)$ where $x == x_i$, and which in addition provides some interpolation of the data $(x_i, y_i)$ for all $x$.

The function `y0 = scipy.interpolate.interp1d(x,y,kind=’..’)` does this interpolation based on splines of varying order. Note that the function `interp1d` returns *a function* `y0` which will then interpolate the x-y data for any given $x$ when called as $y0(x)$.

The code below demonstrates this, and shows the different interpolation kinds.

In [ ]:
@interact(findex=widgets.IntSlider(min=0, max=3, step=1, value=0,layout=widgets.Layout(width='60%')) , 
          Npoints=widgets.IntSlider(min=0, max=100, step=1, value=10,layout=widgets.Layout(width='60%')) , 
          
          continuous_update=True )
def plot(findex,Npoints):


    
        def speciallog(x):

            x[np.where(abs(x)<0.1)]=0.1
            return np.log(x**2)
    
        def func(x):
            if findex == 0:
                return -np.sin(x**2)
            elif findex == 1:
                return x**2
            elif findex == 2:
                return speciallog(x)
            elif findex == 3:
                return np.exp(-0.1*x)*np.sin(0.1*x**2)            

        def create_data(n):
            """Given an integer n, returns n data points
            x and values y as a numpy.array."""
            xmax = 5.
            x = np.linspace(-xmax, xmax, n)
            y = func(x)
            #make y-data somewhat irregular
           # y += 1.5 * np.random.normal(size=len(x))
            return x, y

        #main program
        n = Npoints
        x, y = create_data(n)





        #use finer and regular mesh for plot
        xfine = np.linspace(-4.9, 4.9, n * 100)
        yfine = func(xfine)
        #interpolate with piecewise quadratic (p=2)
        y0 = scipy.interpolate.interp1d(x, y, kind='quadratic')
        #interpolate with piecewise linear func (p=1)
        y1 = scipy.interpolate.interp1d(x, y, kind='linear')
        #interpolate with piecewise constant func (p=3)
        y3 = scipy.interpolate.interp1d(x, y, kind='cubic')
#        print(y3(4.5))
        #interpolate with cubic Hermite
        yfineH = scipy.interpolate.pchip_interpolate(x, y, xfine)

        pylab.figure()
        pylab.plot(x, y, 'o', label='data point')
        pylab.plot(xfine, yfine, label='True function')

        pylab.plot(xfine, y0(xfine), label='quadratic')
        pylab.plot(xfine, y1(xfine), label='linear')
        pylab.plot(xfine, y3(xfine), label='cubic')
        pylab.plot(xfine, yfineH, label='cubic Hermite')
        pylab.legend(loc='upper left',bbox_to_anchor=(1.02,0.5,0.5,0.5))
        pylab.xlabel('x')
        pylab.ylabel('y')

        if findex == 0:
            pylab.text(-4.5,1.25,'f(x) = -sin($x^2$)',fontsize=15)
        elif findex == 1:
            pylab.text(-4.5,27,'f(x) = $x^2$',fontsize=15)
        elif findex == 2:
            pylab.text(-4.5,4,'f(x) = log($x^2$) or $\mathrm{log_e}$(0.01) if abs(x)<0.1',fontsize=15)
        elif findex == 3:
            pylab.text(-4.5,1.6,'f(x) = exp(-0.1x)*sin($x^2$/10) ',fontsize=15)
             

        # plot differences
        pylab.figure()
        pylab.plot(x, y-func(x), 'o', label='data point')
        pylab.plot(xfine, y0(xfine)-func(xfine), label='quadratic')
        pylab.plot(xfine, y1(xfine)-func(xfine), label='linear')
        pylab.plot(xfine, y3(xfine)-func(xfine), label='cubic')
        pylab.plot(xfine, yfineH-func(xfine), label='cubic Hermite')
        pylab.legend(loc='upper left',bbox_to_anchor=(1.02,0.5,0.5,0.5))
        pylab.xlabel('x')
        pylab.ylabel('Interpolation - function')


## Part 2: Root finding

### Root finding using the bisection method

First we introduce the `bisect` algorithm which is (i) robust and (ii) slow but conceptually very simple.

Suppose we need to compute the roots of *f*(*x*)=*x*<sup>3</sup> − 2*x*<sup>2</sup>. This function has a (double) root at *x* = 0 (this is trivial to see) and another root which is located between *x* = 1.5 (where *f*(1.5)= − 1.125) and *x* = 3 (where *f*(3)=9). It is pretty straightforward to see that this other root is located at *x* = 2. Here is a program that determines this root numerically:

In [ ]:
from scipy.optimize import bisect



@interact(xl=widgets.FloatSlider(min=-10, max=2.5, step=0.01, value=-1,layout=widgets.Layout(width='60%')) , 
          xu=widgets.FloatSlider(min=1.9, max=2.5, step=0.01, value=3,layout=widgets.Layout(width='60%')) , 
          tolerance=widgets.IntSlider(min=-10, max=-1, step=1, value=-2,layout=widgets.Layout(width='60%')) , 
          continuous_update=True )
def plot(xl, xu, tolerance):
    
    def f(x):
        """returns f(x)=x^3-2x^2. Has roots at
        x=0 (double root) and x=2"""
        return x ** 3 - 2 * x ** 2


    x=np.linspace(-1,10,1000)
    y=f(x)
    plt.scatter(xl,f(xl),color="red")
    plt.scatter(xu,f(xu),color="blue")

    plt.plot(x,y)
    plt.ylim(np.min(y)-0.2*np.abs( np.min(y)),5)

    # main program starts here
    x = bisect(f, xl, xu, xtol=10**(tolerance))


    plt.text(4,-1, "The root x is approximately {:5.8f}".format(x))
    plt.text(4,-1.5, "The error is {:5.8f}".format(2-x))



The `bisect()` method takes three compulsory arguments: (i) the function *f*(*x*), (ii) a lower limit *a* (for which we have chosen 1.5 in our example) and (ii) an upper limit *b* (for which we have chosen 3). The optional parameter `xtol` determines the maximum error of the method.

### Root finding using Müller-Brent method

This is a classic method to find a zero of the function f on the sign changing interval [a , b]. It is a safe version of the secant method that uses inverse quadratic extrapolation. Brent’s method combines root bracketing, interval bisection, and inverse quadratic interpolation.

In [ ]:
from scipy.optimize import brentq

def f(x):
    return x ** 3 - 2 * x ** 2

x = brentq(f, 1.5, 3, xtol=1e-6)

print("The root x is approximately x=%14.12g,\n"
      "the error is less than 1e-6." % (x))
print("The exact error is %g." % (2 - x))


x = np.linspace(-2,5,1000)
y = f(x)
plt.plot(x,y)
plt.ylim(-3,1)

### Root finding using the `fsolve` function

A (often) better (in the sense of “more efficient”) algorithm than the bisection algorithm is implemented in the general purpose `fsolve()` function for root finding of (multidimensional) functions. This algorithm needs only one starting point close to the suspected location of the root (but is not guaranteed to converge).

Here is an example:

In [ ]:
from scipy.optimize import fsolve

def f(x):
    return x ** 3 - 2 * x ** 2

x0 = [-0.5,1.6]
x = fsolve(f, x0 )           # specify starting points

print("Number of roots is",len(x))
print("The root(s) are ",x)
print("Error of the initial guess = ",x0[0]-0,x0[1]-2)
print("error : ",x[0]-0,x[1]-2)

The input to `fsolve` is the function and the array of initial locations for the roots. The return value of `fsolve` is a numpy array of length *n* for a root finding problem with *n* variables. In the example above, we have *n* = 2.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt


@interact(x0=widgets.FloatSlider(min=-2, max=3, step=0.1, value=-1.8,layout=widgets.Layout(width='60%')) , 
          x1=widgets.FloatSlider(min=-2, max=3, step=0.1, value=3,layout=widgets.Layout(width='60%')) , 
          function=widgets.IntSlider(min=0, max=1, step=1, value=0,layout=widgets.Layout(width='60%')) , 

          continuous_update=True )

def plot(x0,x1,function):
    
    def f(x):
        if function == 0:
            return x ** 3 - 2 * x ** 2
        else:
            return x ** 3 - 2 * x ** 2 + 1
        
    x_initials = [x0,x1]
    if function == 1:
        x_initials = [x0,x1, 0.5]
    x_solutions = fsolve(f,x_initials)
    fig, ax = plt.subplots()    
    plt.scatter( x_initials[0], f(x_initials[0]), color="blue")
    plt.scatter( x_initials[1], f(x_initials[1]), color="blue")
        
    plt.scatter( x_solutions[0], f(x_solutions[0]), color="red")
    plt.scatter( x_solutions[1], f(x_solutions[1]), color="red")
    if function == 1:
        plt.scatter( x_initials[2], f(x_initials[2]), color="blue")
        plt.scatter( x_solutions[2], f(x_solutions[2]), color="red")
        plt.text(0.5,-5,"f(x) = $x^3$ - 2$x^2$ + 1", fontsize=16)
    elif function == 0:
        plt.text(0.5,-5,"f(x) = $x^3$ - 2$x^2$", fontsize=16)
    x=np.linspace(-2,3,100)
    
    

    ax.plot(x, f(x))
    ax.set(xlabel='x', ylabel='f(x)',
           title='Visual check')
    plt.axhline(y=0, c="black")
    plt.axvline(x=0, c="black")
    ax.grid()
    plt.show()